# Sasando Acoustic Replication Pipeline

This notebook implements a comprehensive pipeline to replicate the authentic acoustic sound of the traditional sasando using an electric version.

## Background
The sasando is a traditional Indonesian string instrument where sound is produced by vibrating strings acoustically amplified through a hand-formed, dried lontar leaf resonator. This unique resonator naturally amplifies frequencies between **98 Hz and 1047 Hz**. Modern electric versions replace the resonator with a pickup and speaker, sacrificing acoustic quality for portability.

## Pipeline Overview
1. **Data Collection & Preprocessing** - Load and prepare acoustic and electric recordings
2. **Feature Extraction** - Extract time, frequency, and advanced audio features
3. **Resonator Characterization** - Analyze the acoustic sasando's frequency response
4. **Transformation Design** - Design filters and EQ to match acoustic characteristics
5. **Transformation Application** - Apply transformations to electric signal
6. **Comparison & Validation** - Compute similarity metrics and visualize results
7. **Output** - Save transformed signal and generate reports


## Phase 1: Setup and Imports


In [1]:
# Core libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
from scipy.fft import rfft, rfftfreq, irfft
import warnings
warnings.filterwarnings('ignore')

# Audio analysis libraries
try:
    import librosa
    import librosa.display
    LIBROSA_AVAILABLE = True
except ImportError:
    print("Warning: librosa not available. Some advanced features will be disabled.")
    LIBROSA_AVAILABLE = False

# Audio I/O
try:
    import soundfile as sf
    SOUNDFILE_AVAILABLE = True
except ImportError:
    SOUNDFILE_AVAILABLE = False
    print("Warning: soundfile not available. Using scipy.io.wavfile instead.")

print("Libraries loaded successfully!")
print(f"Librosa available: {LIBROSA_AVAILABLE}")
print(f"Soundfile available: {SOUNDFILE_AVAILABLE}")


Libraries loaded successfully!
Librosa available: True
Soundfile available: True


## Phase 2: Configuration Parameters


In [2]:
# File paths - UPDATE THESE WITH YOUR ACTUAL FILE PATHS
ACOUSTIC_FILE = 'acoustic_sasando.wav'  # Path to acoustic sasando recording
ELECTRIC_FILE = 'electric_sasando.wav'  # Path to electric sasando recording
OUTPUT_FILE = 'transformed_electric_sasando.wav'  # Output file name

# Sasando resonator characteristics
RESONATOR_LOW_FREQ = 98   # Hz - Lower bound of resonator amplification
RESONATOR_HIGH_FREQ = 1047  # Hz - Upper bound of resonator amplification

# Analysis parameters
SAMPLE_RATE = 44100  # Standard audio sample rate (Hz)
FFT_SIZE = 2048      # FFT window size (power of 2)
HOP_LENGTH = 512     # Hop length for spectrogram

# Feature extraction parameters
N_MFCC = 13          # Number of MFCC coefficients
N_CHROMA = 12        # Number of chroma features

# Visualization parameters
MAX_FREQ_PLOT = 2000  # Maximum frequency for plots (Hz)
TIME_SLICE_START = 1.0  # Start time for time slice analysis (seconds)
TIME_SLICE_DURATION = 0.5  # Duration of time slice (seconds)

print("Configuration parameters set!")


Configuration parameters set!


## Phase 3: Data Loading and Preprocessing


In [3]:
def load_audio_file(file_path, target_sr=None):
    """
    Load audio file (WAV or CSV format)
    Returns: audio_data, sample_rate, time_array
    """
    if file_path.endswith('.csv'):
        # Load CSV file (time, amplitude format)
        data = np.genfromtxt(file_path, delimiter=',')
        time = data[:, 0]
        amplitude = data[:, 1]
        sample_rate = 1 / (time[1] - time[0])
        
    elif file_path.endswith('.wav'):
        # Load WAV file
        if SOUNDFILE_AVAILABLE:
            audio_data, sample_rate = sf.read(file_path)
        else:
            sample_rate, audio_data = wavfile.read(file_path)
        
        # Convert to mono if stereo
        if len(audio_data.shape) > 1:
            amplitude = np.mean(audio_data, axis=1)
        else:
            amplitude = audio_data
        
        # Normalize to float32 if needed
        if amplitude.dtype == np.int16:
            amplitude = amplitude.astype(np.float32) / 32768.0
        elif amplitude.dtype == np.int32:
            amplitude = amplitude.astype(np.float32) / 2147483648.0
        
        time = np.arange(0, len(amplitude) / sample_rate, 1 / sample_rate)
    else:
        raise ValueError(f"Unsupported file format: {file_path}")
    
    # Resample if target sample rate is specified
    if target_sr is not None and sample_rate != target_sr:
        if LIBROSA_AVAILABLE:
            amplitude = librosa.resample(amplitude, orig_sr=sample_rate, target_sr=target_sr)
            sample_rate = target_sr
            time = np.arange(0, len(amplitude) / sample_rate, 1 / sample_rate)
        else:
            # Simple resampling using scipy
            num_samples = int(len(amplitude) * target_sr / sample_rate)
            amplitude = signal.resample(amplitude, num_samples)
            sample_rate = target_sr
            time = np.arange(0, len(amplitude) / sample_rate, 1 / sample_rate)
    
    return amplitude, sample_rate, time

def normalize_audio(audio, method='peak'):
    """
    Normalize audio signal
    method: 'peak' (normalize to ±1) or 'rms' (normalize RMS)
    """
    if method == 'peak':
        max_val = np.max(np.abs(audio))
        if max_val > 0:
            return audio / max_val
        return audio
    elif method == 'rms':
        rms = np.sqrt(np.mean(audio**2))
        if rms > 0:
            return audio / rms
        return audio
    return audio

print("Loading functions defined!")


Loading functions defined!


In [4]:
# Load acoustic sasando recording
print("Loading acoustic sasando...")
try:
    acoustic_audio, acoustic_sr, acoustic_time = load_audio_file(ACOUSTIC_FILE, target_sr=SAMPLE_RATE)
    acoustic_audio = normalize_audio(acoustic_audio, method='peak')
    print(f"Acoustic loaded: {len(acoustic_audio)} samples, {len(acoustic_audio)/acoustic_sr:.2f} seconds, {acoustic_sr} Hz")
except FileNotFoundError:
    print(f"Warning: {ACOUSTIC_FILE} not found. Using synthetic data for demonstration.")
    # Generate synthetic acoustic-like signal for demonstration
    duration = 2.0
    t = np.linspace(0, duration, int(SAMPLE_RATE * duration))
    acoustic_audio = np.sin(2 * np.pi * 440 * t) * np.exp(-t * 0.5)  # A4 note with decay
    acoustic_audio += 0.3 * np.sin(2 * np.pi * 880 * t) * np.exp(-t * 0.5)  # Harmonic
    acoustic_audio = normalize_audio(acoustic_audio)
    acoustic_sr = SAMPLE_RATE
    acoustic_time = t

# Load electric sasando recording
print("Loading electric sasando...")
try:
    electric_audio, electric_sr, electric_time = load_audio_file(ELECTRIC_FILE, target_sr=SAMPLE_RATE)
    electric_audio = normalize_audio(electric_audio, method='peak')
    print(f"Electric loaded: {len(electric_audio)} samples, {len(electric_audio)/electric_sr:.2f} seconds, {electric_sr} Hz")
except FileNotFoundError:
    print(f"Warning: {ELECTRIC_FILE} not found. Using synthetic data for demonstration.")
    # Generate synthetic electric-like signal (broader frequency range)
    duration = 2.0
    t = np.linspace(0, duration, int(SAMPLE_RATE * duration))
    electric_audio = np.sin(2 * np.pi * 440 * t) * np.exp(-t * 0.3)
    electric_audio += 0.5 * np.sin(2 * np.pi * 880 * t) * np.exp(-t * 0.3)
    electric_audio += 0.2 * np.sin(2 * np.pi * 1320 * t) * np.exp(-t * 0.3)
    electric_audio += 0.1 * np.sin(2 * np.pi * 2000 * t) * np.exp(-t * 0.3)  # Higher freq
    electric_audio = normalize_audio(electric_audio)
    electric_sr = SAMPLE_RATE
    electric_time = t

print("\nAudio files loaded successfully!")


Loading acoustic sasando...


LibsndfileError: Error opening 'acoustic_sasando.wav': System error.

## Phase 4: Feature Extraction


In [ ]:
def extract_time_domain_features(audio, sr):
    """Extract time domain features"""
    features = {}
    
    # RMS Energy
    features['rms'] = np.sqrt(np.mean(audio**2))
    
    # Zero Crossing Rate
    zero_crossings = np.where(np.diff(np.signbit(audio)))[0]
    features['zcr'] = len(zero_crossings) / (len(audio) / sr)
    
    # Energy
    features['energy'] = np.sum(audio**2)
    
    return features

def extract_frequency_domain_features(audio, sr, n_fft=2048):
    """Extract frequency domain features"""
    features = {}
    
    # FFT
    fft = rfft(audio, n=n_fft)
    freqs = rfftfreq(len(audio), 1/sr)
    magnitude = np.abs(fft)
    
    features['frequencies'] = freqs
    features['magnitude'] = magnitude
    features['power_spectrum'] = magnitude**2
    
    # Spectral centroid (brightness)
    if np.sum(magnitude) > 0:
        features['spectral_centroid'] = np.sum(freqs * magnitude) / np.sum(magnitude)
    else:
        features['spectral_centroid'] = 0
    
    # Spectral rolloff (85% energy)
    cumsum = np.cumsum(magnitude)
    total_energy = cumsum[-1]
    if total_energy > 0:
        rolloff_idx = np.where(cumsum >= 0.85 * total_energy)[0]
        if len(rolloff_idx) > 0:
            features['spectral_rolloff'] = freqs[rolloff_idx[0]]
        else:
            features['spectral_rolloff'] = freqs[-1]
    else:
        features['spectral_rolloff'] = 0
    
    # Spectral bandwidth
    if features['spectral_centroid'] > 0 and np.sum(magnitude) > 0:
        features['spectral_bandwidth'] = np.sqrt(
            np.sum(((freqs - features['spectral_centroid'])**2) * magnitude) / np.sum(magnitude)
        )
    else:
        features['spectral_bandwidth'] = 0
    
    return features

def extract_advanced_features(audio, sr):
    """Extract advanced features using librosa if available"""
    features = {}
    
    if LIBROSA_AVAILABLE:
        # MFCCs (Mel-frequency cepstral coefficients)
        features['mfcc'] = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=N_MFCC)
        
        # Chroma features
        features['chroma'] = librosa.feature.chroma_stft(y=audio, sr=sr, n_chroma=N_CHROMA)
        
        # Spectral contrast
        features['spectral_contrast'] = librosa.feature.spectral_contrast(y=audio, sr=sr)
        
        # Tonnetz (harmonic network)
        features['tonnetz'] = librosa.feature.tonnetz(y=audio, sr=sr)
        
        # Harmonic and percussive components
        features['harmonic'], features['percussive'] = librosa.effects.hpss(audio)
    else:
        print("Librosa not available. Advanced features skipped.")
    
    return features

print("Feature extraction functions defined!")


In [ ]:
# Extract features from acoustic sasando
print("Extracting features from acoustic sasando...")
acoustic_time_features = extract_time_domain_features(acoustic_audio, acoustic_sr)
acoustic_freq_features = extract_frequency_domain_features(acoustic_audio, acoustic_sr, n_fft=FFT_SIZE)
acoustic_advanced_features = extract_advanced_features(acoustic_audio, acoustic_sr)

print(f"Acoustic features:")
print(f"  RMS: {acoustic_time_features['rms']:.4f}")
print(f"  ZCR: {acoustic_time_features['zcr']:.2f} Hz")
print(f"  Spectral Centroid: {acoustic_freq_features['spectral_centroid']:.2f} Hz")
print(f"  Spectral Rolloff: {acoustic_freq_features['spectral_rolloff']:.2f} Hz")
print(f"  Spectral Bandwidth: {acoustic_freq_features['spectral_bandwidth']:.2f} Hz")

# Extract features from electric sasando
print("\nExtracting features from electric sasando...")
electric_time_features = extract_time_domain_features(electric_audio, electric_sr)
electric_freq_features = extract_frequency_domain_features(electric_audio, electric_sr, n_fft=FFT_SIZE)
electric_advanced_features = extract_advanced_features(electric_audio, electric_sr)

print(f"Electric features:")
print(f"  RMS: {electric_time_features['rms']:.4f}")
print(f"  ZCR: {electric_time_features['zcr']:.2f} Hz")
print(f"  Spectral Centroid: {electric_freq_features['spectral_centroid']:.2f} Hz")
print(f"  Spectral Rolloff: {electric_freq_features['spectral_rolloff']:.2f} Hz")
print(f"  Spectral Bandwidth: {electric_freq_features['spectral_bandwidth']:.2f} Hz")


## Phase 5: Resonator Characterization


In [ ]:
def analyze_resonator_response(audio, sr, low_freq, high_freq, n_fft=2048):
    """
    Analyze the frequency response of the resonator
    Returns frequency response within the resonator range
    """
    # Compute frequency response
    fft = rfft(audio, n=n_fft)
    freqs = rfftfreq(len(audio), 1/sr)
    magnitude = np.abs(fft)
    
    # Filter to resonator frequency range
    mask = (freqs >= low_freq) & (freqs <= high_freq)
    resonator_freqs = freqs[mask]
    resonator_magnitude = magnitude[mask]
    
    # Normalize magnitude
    if np.max(resonator_magnitude) > 0:
        resonator_magnitude_norm = resonator_magnitude / np.max(resonator_magnitude)
    else:
        resonator_magnitude_norm = resonator_magnitude
    
    # Find dominant frequencies (peaks)
    from scipy.signal import find_peaks
    peaks, properties = find_peaks(resonator_magnitude_norm, height=0.1, distance=10)
    peak_freqs = resonator_freqs[peaks]
    peak_magnitudes = resonator_magnitude_norm[peaks]
    
    return {
        'frequencies': resonator_freqs,
        'magnitude': resonator_magnitude,
        'magnitude_norm': resonator_magnitude_norm,
        'peak_frequencies': peak_freqs,
        'peak_magnitudes': peak_magnitudes,
        'mean_gain': np.mean(resonator_magnitude_norm),
        'max_gain': np.max(resonator_magnitude_norm)
    }

# Analyze acoustic resonator response
print("Analyzing acoustic resonator response...")
acoustic_resonator = analyze_resonator_response(
    acoustic_audio, acoustic_sr, 
    RESONATOR_LOW_FREQ, RESONATOR_HIGH_FREQ, 
    n_fft=FFT_SIZE
)

print(f"Resonator frequency range: {RESONATOR_LOW_FREQ} - {RESONATOR_HIGH_FREQ} Hz")
print(f"Mean gain in range: {acoustic_resonator['mean_gain']:.4f}")
print(f"Max gain in range: {acoustic_resonator['max_gain']:.4f}")
print(f"Number of peaks found: {len(acoustic_resonator['peak_frequencies'])}")
if len(acoustic_resonator['peak_frequencies']) > 0:
    print(f"Dominant frequencies: {acoustic_resonator['peak_frequencies'][:5]} Hz")


## Phase 6: Transformation Design


In [ ]:
def design_bandpass_filter(low_freq, high_freq, sr, order=4):
    """Design a bandpass filter for the resonator frequency range"""
    nyquist = sr / 2
    low = low_freq / nyquist
    high = high_freq / nyquist
    
    # Ensure frequencies are within valid range
    low = max(0.01, min(low, 0.99))
    high = max(0.01, min(high, 0.99))
    
    b, a = signal.butter(order, [low, high], btype='band')
    return b, a

def design_eq_curve(reference_freq_response, target_freq_response, freqs):
    """
    Design an EQ curve to match target frequency response
    Returns gain values for each frequency
    """
    # Normalize both responses
    ref_norm = reference_freq_response / (np.max(reference_freq_response) + 1e-10)
    target_norm = target_freq_response / (np.max(target_freq_response) + 1e-10)
    
    # Calculate gain needed (in dB)
    # Avoid division by zero
    ref_norm = np.clip(ref_norm, 1e-10, None)
    target_norm = np.clip(target_norm, 1e-10, None)
    
    gain_db = 20 * np.log10(target_norm / ref_norm)
    
    # Smooth the EQ curve to avoid artifacts
    from scipy.ndimage import gaussian_filter1d
    gain_db_smooth = gaussian_filter1d(gain_db, sigma=2)
    
    return gain_db_smooth, gain_db

def apply_eq_curve(audio, sr, freqs, gain_db, n_fft=2048):
    """Apply EQ curve to audio signal"""
    # Compute FFT
    fft = rfft(audio, n=n_fft)
    audio_freqs = rfftfreq(len(audio), 1/sr)
    
    # Interpolate gain curve to match audio frequencies
    from scipy.interpolate import interp1d
    gain_interp = interp1d(freqs, gain_db, kind='linear', 
                          bounds_error=False, fill_value=0)
    gain_at_audio_freqs = gain_interp(audio_freqs)
    
    # Convert dB to linear gain
    gain_linear = 10**(gain_at_audio_freqs / 20)
    
    # Apply gain
    fft_eq = fft * gain_linear
    
    # Convert back to time domain
    audio_eq = irfft(fft_eq, n=len(audio))
    
    return audio_eq

print("Transformation design functions defined!")


In [ ]:
# Design bandpass filter
print("Designing bandpass filter...")
bp_b, bp_a = design_bandpass_filter(RESONATOR_LOW_FREQ, RESONATOR_HIGH_FREQ, electric_sr)
print(f"Bandpass filter: {RESONATOR_LOW_FREQ} - {RESONATOR_HIGH_FREQ} Hz")

# Design EQ curve based on acoustic resonator response
print("\nDesigning EQ curve...")
# Get electric frequency response in resonator range
electric_resonator = analyze_resonator_response(
    electric_audio, electric_sr,
    RESONATOR_LOW_FREQ, RESONATOR_HIGH_FREQ,
    n_fft=FFT_SIZE
)

# Design EQ to match acoustic response
eq_gain_db, eq_gain_db_raw = design_eq_curve(
    electric_resonator['magnitude'],
    acoustic_resonator['magnitude'],
    electric_resonator['frequencies']
)

print(f"EQ curve designed. Gain range: {np.min(eq_gain_db):.2f} to {np.max(eq_gain_db):.2f} dB")
print("Transformation pipeline designed!")


## Phase 7: Transformation Application


In [ ]:
# Apply transformations to electric signal
print("Applying transformations to electric signal...")

# Step 1: Apply bandpass filter
print("  Step 1: Applying bandpass filter...")
electric_filtered = signal.filtfilt(bp_b, bp_a, electric_audio)

# Step 2: Apply EQ curve
print("  Step 2: Applying EQ curve...")
electric_eq = apply_eq_curve(
    electric_filtered, electric_sr,
    electric_resonator['frequencies'],
    eq_gain_db,
    n_fft=FFT_SIZE
)

# Step 3: Normalize output
print("  Step 3: Normalizing output...")
transformed_audio = normalize_audio(electric_eq, method='peak')

print("Transformation complete!")
print(f"Original RMS: {np.sqrt(np.mean(electric_audio**2)):.4f}")
print(f"Transformed RMS: {np.sqrt(np.mean(transformed_audio**2)):.4f}")


## Phase 8: Similarity Metrics Computation


In [ ]:
def compute_euclidean_distance(vec1, vec2):
    """Compute Euclidean distance between two vectors"""
    # Ensure same length
    min_len = min(len(vec1), len(vec2))
    return np.sqrt(np.sum((vec1[:min_len] - vec2[:min_len])**2))

def compute_cosine_similarity(vec1, vec2):
    """Compute cosine similarity between two vectors"""
    # Ensure same length
    min_len = min(len(vec1), len(vec2))
    v1 = vec1[:min_len]
    v2 = vec2[:min_len]
    
    dot_product = np.dot(v1, v2)
    norm1 = np.linalg.norm(v1)
    norm2 = np.linalg.norm(v2)
    
    if norm1 == 0 or norm2 == 0:
        return 0.0
    
    return dot_product / (norm1 * norm2)

def compute_mfcc_distance(mfcc1, mfcc2):
    """Compute distance between MFCC feature matrices"""
    if not LIBROSA_AVAILABLE:
        return None
    
    # Average over time dimension
    mfcc1_mean = np.mean(mfcc1, axis=1)
    mfcc2_mean = np.mean(mfcc2, axis=1)
    
    return compute_euclidean_distance(mfcc1_mean, mfcc2_mean)

def compute_spectral_distance(freq_features1, freq_features2):
    """Compute spectral distance metrics"""
    # Interpolate to common frequency grid
    from scipy.interpolate import interp1d
    
    freqs1 = freq_features1['frequencies']
    mag1 = freq_features1['magnitude']
    freqs2 = freq_features2['frequencies']
    mag2 = freq_features2['magnitude']
    
    # Common frequency range
    min_freq = max(freqs1[0], freqs2[0])
    max_freq = min(freqs1[-1], freqs2[-1])
    common_freqs = np.linspace(min_freq, max_freq, 1000)
    
    # Interpolate
    interp1 = interp1d(freqs1, mag1, kind='linear', bounds_error=False, fill_value=0)
    interp2 = interp1d(freqs2, mag2, kind='linear', bounds_error=False, fill_value=0)
    
    mag1_interp = interp1(common_freqs)
    mag2_interp = interp2(common_freqs)
    
    # Normalize
    mag1_norm = mag1_interp / (np.max(mag1_interp) + 1e-10)
    mag2_norm = mag2_interp / (np.max(mag2_interp) + 1e-10)
    
    # Compute metrics
    euclidean = compute_euclidean_distance(mag1_norm, mag2_norm)
    cosine = compute_cosine_similarity(mag1_norm, mag2_norm)
    
    return {
        'euclidean_distance': euclidean,
        'cosine_similarity': cosine
    }

print("Similarity metric functions defined!")


In [ ]:
# Extract features from transformed signal
print("Extracting features from transformed signal...")
transformed_freq_features = extract_frequency_domain_features(transformed_audio, electric_sr, n_fft=FFT_SIZE)
transformed_advanced_features = extract_advanced_features(transformed_audio, electric_sr)

# Compute similarity metrics
print("\nComputing similarity metrics...")

# Spectral distance
spectral_metrics = compute_spectral_distance(acoustic_freq_features, transformed_freq_features)
print(f"Spectral Euclidean Distance: {spectral_metrics['euclidean_distance']:.4f}")
print(f"Spectral Cosine Similarity: {spectral_metrics['cosine_similarity']:.4f}")

# MFCC distance
if LIBROSA_AVAILABLE and 'mfcc' in acoustic_advanced_features and 'mfcc' in transformed_advanced_features:
    mfcc_dist = compute_mfcc_distance(
        acoustic_advanced_features['mfcc'],
        transformed_advanced_features['mfcc']
    )
    if mfcc_dist is not None:
        print(f"MFCC Distance: {mfcc_dist:.4f}")

# Compare with original electric
print("\nComparing with original electric signal:")
electric_spectral_metrics = compute_spectral_distance(acoustic_freq_features, electric_freq_features)
print(f"Original Electric - Spectral Euclidean Distance: {electric_spectral_metrics['euclidean_distance']:.4f}")
print(f"Original Electric - Spectral Cosine Similarity: {electric_spectral_metrics['cosine_similarity']:.4f}")


## Phase 9: Visualization


In [ ]:
# Create comprehensive visualization
fig = plt.figure(figsize=(16, 12))

# 1. Time domain waveforms
ax1 = plt.subplot(3, 3, 1)
time_slice = int(TIME_SLICE_START * acoustic_sr)
time_slice_end = int((TIME_SLICE_START + TIME_SLICE_DURATION) * acoustic_sr)
ax1.plot(acoustic_time[time_slice:time_slice_end], acoustic_audio[time_slice:time_slice_end], 'b-', label='Acoustic', alpha=0.7)
ax1.plot(electric_time[time_slice:time_slice_end], electric_audio[time_slice:time_slice_end], 'r-', label='Electric (Original)', alpha=0.7)
ax1.plot(electric_time[time_slice:time_slice_end], transformed_audio[time_slice:time_slice_end], 'g-', label='Electric (Transformed)', alpha=0.7)
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Amplitude')
ax1.set_title('Time Domain Waveforms (Slice)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Frequency response comparison
ax2 = plt.subplot(3, 3, 2)
mask = acoustic_freq_features['frequencies'] <= MAX_FREQ_PLOT
ax2.plot(acoustic_freq_features['frequencies'][mask], acoustic_freq_features['magnitude'][mask], 'b-', label='Acoustic', alpha=0.7, linewidth=2)
mask = electric_freq_features['frequencies'] <= MAX_FREQ_PLOT
ax2.plot(electric_freq_features['frequencies'][mask], electric_freq_features['magnitude'][mask], 'r-', label='Electric (Original)', alpha=0.7)
mask = transformed_freq_features['frequencies'] <= MAX_FREQ_PLOT
ax2.plot(transformed_freq_features['frequencies'][mask], transformed_freq_features['magnitude'][mask], 'g-', label='Electric (Transformed)', alpha=0.7)
ax2.axvspan(RESONATOR_LOW_FREQ, RESONATOR_HIGH_FREQ, alpha=0.2, color='yellow', label='Resonator Range')
ax2.set_xlabel('Frequency (Hz)')
ax2.set_ylabel('Magnitude')
ax2.set_title('Frequency Response Comparison')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_xlim([0, MAX_FREQ_PLOT])

# 3. Resonator frequency range detail
ax3 = plt.subplot(3, 3, 3)
ax3.plot(acoustic_resonator['frequencies'], acoustic_resonator['magnitude_norm'], 'b-', label='Acoustic', linewidth=2)
ax3.plot(electric_resonator['frequencies'], electric_resonator['magnitude_norm'], 'r-', label='Electric (Original)', alpha=0.7)
transformed_resonator = analyze_resonator_response(transformed_audio, electric_sr, RESONATOR_LOW_FREQ, RESONATOR_HIGH_FREQ, n_fft=FFT_SIZE)
ax3.plot(transformed_resonator['frequencies'], transformed_resonator['magnitude_norm'], 'g-', label='Electric (Transformed)', alpha=0.7)
ax3.set_xlabel('Frequency (Hz)')
ax3.set_ylabel('Normalized Magnitude')
ax3.set_title('Resonator Range (98-1047 Hz)')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. EQ Curve
ax4 = plt.subplot(3, 3, 4)
ax4.plot(electric_resonator['frequencies'], eq_gain_db, 'purple', linewidth=2)
ax4.axhline(0, color='black', linestyle='--', alpha=0.5)
ax4.set_xlabel('Frequency (Hz)')
ax4.set_ylabel('Gain (dB)')
ax4.set_title('EQ Curve (Applied to Electric)')
ax4.grid(True, alpha=0.3)

# 5. Spectrograms - Acoustic
ax5 = plt.subplot(3, 3, 5)
freqs_ac, times_ac, Sx_ac = signal.spectrogram(acoustic_audio, fs=acoustic_sr, nperseg=1024, noverlap=512)
mask_freq = freqs_ac <= MAX_FREQ_PLOT
im5 = ax5.pcolormesh(times_ac, freqs_ac[mask_freq], 10 * np.log10(Sx_ac[mask_freq, :] + 1e-10), cmap='viridis', shading='auto')
ax5.set_ylabel('Frequency (Hz)')
ax5.set_xlabel('Time (s)')
ax5.set_title('Acoustic Spectrogram')
plt.colorbar(im5, ax=ax5, label='Power (dB)')

# 6. Spectrograms - Electric Original
ax6 = plt.subplot(3, 3, 6)
freqs_el, times_el, Sx_el = signal.spectrogram(electric_audio, fs=electric_sr, nperseg=1024, noverlap=512)
mask_freq = freqs_el <= MAX_FREQ_PLOT
im6 = ax6.pcolormesh(times_el, freqs_el[mask_freq], 10 * np.log10(Sx_el[mask_freq, :] + 1e-10), cmap='viridis', shading='auto')
ax6.set_ylabel('Frequency (Hz)')
ax6.set_xlabel('Time (s)')
ax6.set_title('Electric Original Spectrogram')
plt.colorbar(im6, ax=ax6, label='Power (dB)')

# 7. Spectrograms - Electric Transformed
ax7 = plt.subplot(3, 3, 7)
freqs_tr, times_tr, Sx_tr = signal.spectrogram(transformed_audio, fs=electric_sr, nperseg=1024, noverlap=512)
mask_freq = freqs_tr <= MAX_FREQ_PLOT
im7 = ax7.pcolormesh(times_tr, freqs_tr[mask_freq], 10 * np.log10(Sx_tr[mask_freq, :] + 1e-10), cmap='viridis', shading='auto')
ax7.set_ylabel('Frequency (Hz)')
ax7.set_xlabel('Time (s)')
ax7.set_title('Electric Transformed Spectrogram')
plt.colorbar(im7, ax=ax7, label='Power (dB)')

# 8. Spectral features comparison
ax8 = plt.subplot(3, 3, 8)
features = ['Centroid', 'Rolloff', 'Bandwidth']
acoustic_vals = [acoustic_freq_features['spectral_centroid'], 
                 acoustic_freq_features['spectral_rolloff'], 
                 acoustic_freq_features['spectral_bandwidth']]
electric_vals = [electric_freq_features['spectral_centroid'], 
                electric_freq_features['spectral_rolloff'], 
                electric_freq_features['spectral_bandwidth']]
transformed_vals = [transformed_freq_features['spectral_centroid'], 
                   transformed_freq_features['spectral_rolloff'], 
                   transformed_freq_features['spectral_bandwidth']]
x = np.arange(len(features))
width = 0.25
ax8.bar(x - width, acoustic_vals, width, label='Acoustic', color='blue', alpha=0.7)
ax8.bar(x, electric_vals, width, label='Electric (Original)', color='red', alpha=0.7)
ax8.bar(x + width, transformed_vals, width, label='Electric (Transformed)', color='green', alpha=0.7)
ax8.set_ylabel('Frequency (Hz)')
ax8.set_title('Spectral Features Comparison')
ax8.set_xticks(x)
ax8.set_xticklabels(features)
ax8.legend()
ax8.grid(True, alpha=0.3, axis='y')

# 9. Similarity metrics
ax9 = plt.subplot(3, 3, 9)
metrics_names = ['Euclidean\nDistance', 'Cosine\nSimilarity']
original_metrics = [electric_spectral_metrics['euclidean_distance'], 
                    electric_spectral_metrics['cosine_similarity']]
transformed_metrics = [spectral_metrics['euclidean_distance'], 
                      spectral_metrics['cosine_similarity']]
x = np.arange(len(metrics_names))
width = 0.35
ax9.bar(x - width/2, original_metrics, width, label='Original Electric', color='red', alpha=0.7)
ax9.bar(x + width/2, transformed_metrics, width, label='Transformed Electric', color='green', alpha=0.7)
ax9.set_ylabel('Metric Value')
ax9.set_title('Similarity Metrics vs Acoustic')
ax9.set_xticks(x)
ax9.set_xticklabels(metrics_names)
ax9.legend()
ax9.grid(True, alpha=0.3, axis='y')
ax9.axhline(1.0, color='black', linestyle='--', alpha=0.5, label='Perfect Match')

plt.tight_layout()
plt.savefig('sasando_analysis_results.png', dpi=150, bbox_inches='tight')
plt.show()

print("Visualization complete! Saved as 'sasando_analysis_results.png'")


In [ ]:
# Save transformed audio
print(f"Saving transformed audio to {OUTPUT_FILE}...")

# Convert to int16 for WAV file
transformed_audio_int16 = (transformed_audio * 32767).astype(np.int16)

try:
    if SOUNDFILE_AVAILABLE:
        sf.write(OUTPUT_FILE, transformed_audio_int16, electric_sr)
    else:
        wavfile.write(OUTPUT_FILE, electric_sr, transformed_audio_int16)
    print(f"Successfully saved to {OUTPUT_FILE}")
except Exception as e:
    print(f"Error saving file: {e}")

# Generate summary report
print("\n" + "="*60)
print("ANALYSIS SUMMARY")
print("="*60)
print(f"\nResonator Frequency Range: {RESONATOR_LOW_FREQ} - {RESONATOR_HIGH_FREQ} Hz")
print(f"\nSimilarity Metrics (vs Acoustic):")
print(f"  Transformed Electric:")
print(f"    - Euclidean Distance: {spectral_metrics['euclidean_distance']:.4f}")
print(f"    - Cosine Similarity: {spectral_metrics['cosine_similarity']:.4f}")
print(f"  Original Electric:")
print(f"    - Euclidean Distance: {electric_spectral_metrics['euclidean_distance']:.4f}")
print(f"    - Cosine Similarity: {electric_spectral_metrics['cosine_similarity']:.4f}")
print(f"\nImprovement:")
euclidean_improvement = ((electric_spectral_metrics['euclidean_distance'] - spectral_metrics['euclidean_distance']) / 
                         electric_spectral_metrics['euclidean_distance'] * 100)
cosine_improvement = ((spectral_metrics['cosine_similarity'] - electric_spectral_metrics['cosine_similarity']) / 
                     (1 - electric_spectral_metrics['cosine_similarity'] + 1e-10) * 100)
print(f"  - Euclidean Distance: {euclidean_improvement:.2f}% reduction (lower is better)")
print(f"  - Cosine Similarity: {cosine_improvement:.2f}% improvement (higher is better)")
print("\n" + "="*60)
